## How to run your customized workflow?

### Requirement
You need to have an OpenAI API Key.

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "Your API Key"

### Set overall task & Customize your own workflow
Firstly, you need to define your overall task. After that, you can freely define your own workflow.  
Following attributes need to be provided for each subtask:
- **`id`**: The identifier for this subtask.  
  Format: `"taskx"`, where `x` is an Arabic numeral (e.g., `task1`, `task2`).

- **`objective`**: The specific content or goal of this subtask.

- **`agent_id`**: The identifier for the agent responsible for executing this subtask.

- **`next`**: A list of tasks that depend on the completion of this subtask.  
  Values in this list must be `id`s of previously defined tasks.

- **`prev`**: A list of tasks that this subtask depends on.  
  Values in this list must be `id`s of previously defined tasks.

- **`agent`**: The name of the agent responsible for completing this subtask.  
  The same `agent_id` must always correspond to the same `agent`.

Note That：
- Your workflow **must not** contain any form of circular dependencies.
- Ensure that all `next` and `prev` values reference valid `id`s of previously defined tasks.
- The `agent_id` and `agent` must remain consistent across all tasks.

In [ ]:
from workflow import Task, Workflow

# Customize your overall task here.
overall_task: str = "Your own task"

# Use a list of dictionaries to initialize task data here.
tasks_data = [
    {
        "id": "task0",
        "objective": "objective0",
        "agent_id": 0,
        "next": ["task1"],
        "prev": [],
        "agent": "agent0",
    },
    {
        "id": "task1",
        "objective": "objective1",
        "agent_id": 1,
        "next": [],
        "prev": ["task0"],
        "agent": "agent1",
    },
]

# Calculate dependencies and create a dictionary of tasks
tasks_dic = {task_data["id"]: Task(**task_data) for task_data in tasks_data}

# Create the workflow
customized_workflow = Workflow(tasks_dic)

### Run your workflow

In [ ]:
import asyncio
import nest_asyncio
import json
import time
from flow import Flow
import logging
from summary import Summary
# -----------------------------------------------------------------------------
# Configuration and Logging Setup
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# Entry point for running the workflow. Creates an initial workflow and orchestrates the manager.

start_time = time.time()

manager = Flow(overall_task=overall_task, enable_refine=False, refine_threhold=0, n_candidate_graphs=0, customize=True, customize_workflow=customized_workflow)
nest_asyncio.apply()
asyncio.run(manager.run_async())

elapsed_time = time.time() - start_time
logger.info(f"Elapsed time: {elapsed_time:.2f} seconds")

workflow_data = {
    tid: task.__dict__ for tid, task in manager.workflow.tasks.items()
}
with open('manually_result.json', 'w', encoding='utf-8') as file:
    json.dump(workflow_data, file, indent=4)

summary = Summary()

# Generate and save a summary of the workflow results
chat_result = summary.summary(overall_task, workflow_data)
print(chat_result)